In [1]:
import numpy as np
import pandas as pd
import multiprocessing

import scipy
from scipy import spatial
from scipy.spatial import cKDTree

import sklearn as sk
from sklearn import svm
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn import clone
from sklearn.externals.six.moves import xrange

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline

%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

Loading BokehJS ...

In [ ]:
# 2018: SS18/FW18 ranges
# SS18 = pd.read_excel('RangeSS18.xlsx', skiprows = 5) # **PROBLEM**
SS18 = pd.read_excel('SS18_range_data.xlsx') # Solution: Artem file
FW18 = pd.read_excel('RangeFW18.xlsx', skiprows = 5)

SS18_range = SS18['Article Number'].unique()
FW18_range = FW18['Article Number'].unique()

In [ ]:
# SS20 = (pd.read_excel('SS20Range.xlsx', skiprows = 6).
#        drop(['Unnamed: 0', 'vlookup'], axis = 1))

# SS20_all_articles = SS20['Article Number'].unique()

# SS20 = SS20[SS20['CO'] == 'Y']
# dat['CO'].fillna('N', inplace = True)
# dat['SS19 Ranged'].fillna('N', inplace = True)
# dat['FW19 Ranged'].fillna('N', inplace = True)

# dat.to_csv('dat_SS20_range.csv')

dat_SS20_range = pd.read_csv('dat_SS20_range.csv') # *** DATA ***
SS20_range = dat_SS20_range['Article Number'].unique() # 1340

In [ ]:
# EDA
dat_SS20_range = dat_SS20_range[['Article Number', 'Model', 'Product Division', 
                                 'Category Marketing Line', 'Gender', 'Model Number', 'Product Group',
                                 'Product Type', 'Sports Category', 'Key Category', 'WE eCom', 'SS19 Ranged', 
                                 'FW19 Ranged', 'CO']]


In [ ]:
dat_SS20_range.groupby(['SS19 Ranged', 'FW19 Ranged', 'CO'])['Article Number'].count()
# dat_SS20_range['Key Category'].value_counts()[:10]
# dat_SS20_range['Product Division'].value_counts()
# dat_SS20_range['Product Group'].value_counts()[:10]
# dat_SS20_range['Gender'].value_counts()

# dat_SS20_range[dat_SS20_range['FW19 Ranged'] == 'Y'].groupby('Article Number')[]

## Corrections

In [ ]:
# # ----- Benoit's magic ----
# dat.loc[dat.order_date.apply(lambda x: "mkt-Raiffeisen Bank" in x),:].index

In [2]:
# Wall time: 2min 9s
dat0 = pd.read_csv('Transaction data.csv', low_memory=False) # *** DATA ***
dat0['consumer_order_date'] = pd.to_datetime(dat0['consumer_order_date'])

In [3]:
dat = dat0.copy()

In [ ]:
dat.groupby('season')['Clearance'].value_counts()

In [4]:
dat = dat[['consumer_order_date', 'article_number', 'gross_demand_quantity', 'Clearance', 'season']]
dat = dat[dat['season'] == 'SS19']

In [ ]:
# ------ SHENANIGANS ------

# all_SS19_articles = dat_SS19['article_number'].unique() # SS19 article count

# dat_SS20_range = pd.read_csv('dat_SS20_range.csv')
# SS20_range = dat_SS20_range['Article Number'].unique()

# len(set(all_SS19_articles).intersection(set(SS20_range)))
# dat_SS19_SS20 = dat_SS19[[(a in SS20_range) for a in dat_SS19['article_number']]]

# dat_SS19_season_SS20 = pd.DataFrame(dat_SS19_SS20.groupby('article_number')['gross_demand_quantity'].sum())

# SS20_carryover_gt83 = dat_SS19_season_SS20[dat_SS19_season_SS20['gross_demand_quantity'] > 83]

In [5]:
# Wall time: 26.2 s

dat_SS20_range = pd.read_csv('dat_SS20_range.csv') # *** DATA ***
SS20_range = dat_SS20_range['Article Number'].unique()
dat = dat[[(a in SS20_range) for a in dat['article_number']]]

In [6]:
clearance = dat.copy() # FOR CLEARANCE CORRECTION IN NEXT SECTION

# 'aggregate' to weekly sums by article for buy_availability merge and adjustment
dat.set_index('consumer_order_date', inplace = True)

dat = dat[['article_number', 'gross_demand_quantity']].groupby(['article_number']).resample('W').sum()
dat.reset_index(inplace=True)

# Add 'week' and 'year' for merging with stock (buy_availability) data (b/c min_date_of_week)
dat['week'] = [t.week for t in dat['consumer_order_date']]
dat['year'] = [t.year for t in dat['consumer_order_date']]

In [7]:
# Wall time: ~40 s

# SS19 in Dec. 2018 -- to merge with 2019 SS19
dat_stock = pd.read_csv('Stock Data/Stock data 2018.csv', low_memory=False, index_col = 0) # *** DATA ***
dat_stock['min_date_of_week'] = pd.to_datetime(dat_stock['min_date_of_week'])
dat_stock = dat_stock[dat_stock['min_date_of_week'] > pd.to_datetime('2018-12-1')] # filter to > Dec1 (SS19)

# 2019 data
dat_stock2 = pd.read_csv('Stock Data/Stock data 2019.csv', low_memory=False, index_col = 0) # *** DATA ***

# Build: Dec2018 + 2019
dat_stock = pd.concat([dat_stock, dat_stock2])

# Tidy
dat_stock.reset_index(inplace = True)
dat_stock.drop(['avg(ecom_available_stock)', 'avg(size_availability)'], axis = 1, inplace = True)
dat_stock.rename(columns = {'min_date_of_week': 'date', 'avg(buy_availability)': 'buy_availability'}, inplace = True)

# Filter to SS20_range carryovers
dat_stock = dat_stock[[(a in SS20_range) for a in dat_stock['article_number']]]

dat_stock['date'] = pd.to_datetime(dat_stock['date'])

# For merging with transaction data
dat_stock['week'] = [t.week for t in dat_stock['date']]
dat_stock['year'] = [t.year for t in dat_stock['date']]

In [8]:
# weekly demand + buy_availability -- for understock correction
dat = pd.merge(dat, dat_stock, 
               left_on = ['article_number', 'year', 'week'], 
               right_on = ['article_number', 'year', 'week'],
               how = 'outer')

dat['buy_availability'] = dat['buy_availability'].fillna(1)

In [9]:
# ----- Calculate *observed* full season gross_qty per article -----
dat_season = pd.DataFrame(dat.groupby(['article_number'])['gross_demand_quantity'].sum())

dat_season.rename(columns = {'gross_demand_quantity':'season_gross_demand_quantity'}, inplace = True)

In [10]:
# WEEKLY averages for articles when fully stocked
dat_stocked = pd.DataFrame(
    dat[dat['buy_availability'] > 0.35].
    groupby(['article_number'])['gross_demand_quantity'].
    mean())

dat_stocked.rename(columns = {'gross_demand_quantity':'stocked_weekly_avg_gross_demand_quantity'}, inplace= True)

dat_stocked['understock_correction'] = 26*dat_stocked['stocked_weekly_avg_gross_demand_quantity']

In [12]:
dat_season = pd.merge(dat_season, dat_stocked, left_index=True, right_index=True, how = 'outer')
dat_season.drop('stocked_weekly_avg_gross_demand_quantity', inplace=True, axis = 1)

In [ ]:
# ------ OVERSTOCK CORRECTION ------

In [13]:
clearance0 = clearance[clearance['Clearance'] == 0]

clearance0 = pd.DataFrame(clearance0.groupby(['article_number'])['gross_demand_quantity'].sum())

clearance0.rename(columns = {'gross_demand_quantity': 'overstock_correction'}, inplace = True)

(268593, 5)

In [16]:
dat_season = pd.merge(dat_season, clearance0, how = 'outer', left_index = True, right_index = True)

In [18]:
preds = (pd.DataFrame(
    pd.read_csv('Buyers predictions.csv', low_memory=False, index_col = 0)) # *** DATA ***
         [['season', 'ecom_marketing_forecast']].
         reset_index().
        dropna())

preds = preds[preds['season'] == 'SS19']

dat_season = pd.merge(
    dat_season, preds, 
    left_on = ['article_number'],
    right_on=['article'], 
    how = 'left').round()

dat_season = dat_season[['article', 'season', 'season_gross_demand_quantity', 'understock_correction',
                        'overstock_correction', 'ecom_marketing_forecast']]

In [29]:
def projection(gross, forecast, under, over):
    if gross < forecast:
        return under
    elif gross > forecast:
        return over
    else:
        return gross

dat_season.loc[:,'projected_gross_demand_quantity'] = (dat_season.
                          apply(lambda row: projection(row['season_gross_demand_quantity'], 
                                                       row['ecom_marketing_forecast'], 
                                                       row['understock_correction'], 
                                                       row['overstock_correction']), 
                                axis = 1))

In [27]:
dat_season

,article,season,season_gross_demand_quantity,understock_correction,overstock_correction,ecom_marketing_forecast,projected_gross_demand_quantity
0,011040,SS19,52.0,68.0,52.0,600.0,68.0
1,015110,SS19,280.0,364.0,280.0,800.0,364.0
2,019000,SS19,693.0,901.0,693.0,2000.0,901.0
3,019228,SS19,174.0,226.0,174.0,500.0,226.0
4,019310,SS19,129.0,168.0,129.0,500.0,168.0
5,033200,SS19,44.0,64.0,44.0,300.0,64.0
6,033201,SS19,71.0,123.0,71.0,600.0,123.0
7,033905,SS19,498.0,647.0,498.0,1500.0,647.0
8,218977,SS19,71.0,97.0,71.0,247.0,97.0
9,279008,SS19,21.0,78.0,21.0,256.0,78.0


In [ ]:
# Chris, could you give me over/understock correction for 'B41674'
# Also: CE2441
